In [8]:
pip install tf-keras

In [1]:
# 📌 Step 2: Import Libraries
import psycopg2
import pandas as pd
import os

# 📌 Step 3: Set Up Database Connection (Replace with Your Details)
DATABASE_URL = "dbname='postgres' user='postgres' host='34.59.119.208' password='avantichhaya'"

def connect_db():
    try:
        conn = psycopg2.connect(DATABASE_URL)
        print("✅ Connected to PostgreSQL Cloud SQL")
        return conn
    except Exception as e:
        print("🚨 Connection Error:", e)
        return None

conn = connect_db()
cursor = conn.cursor()


✅ Connected to PostgreSQL Cloud SQL


cursor.execute("""
    UPDATE emails
    SET classified_category = NULL,
        escalated = NULL,
        classification_confidence = NULL
    WHERE email_id >= 101 AND email_id <= 180;
""")
conn.commit()

In [2]:
from transformers import pipeline

# Step 1: Setup classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Step 2: Define labels
label_map = {
    "Leave Request": "Requests related to taking leave or vacation",
    "Onboarding": "Questions about new hire onboarding or joining formalities",
    "Job Offer": "Inquiries regarding job offers or employment contracts",
    "Payroll Inquiry": "Questions related to salary, payslips, or payroll processing",
    "Benefits Inquiry": "Questions about insurance, medical, or employee benefits",
    "Resignation & Exit": "Emails about resignation, exit process, or final settlements",
    "Attendance & Timesheet": "Issues about work hours, attendance or timesheets",
    "Recruitment Process": "Questions about interview, screening or hiring stages",
    "Policy Clarification": "Clarification about company policies or procedures",
    "Training & Development": "Queries about training programs or skill development",
    "Work From Home Requests": "Requests or updates regarding remote work",
    "Relocation & Transfer": "Inquiries about internal transfers or relocation",
    "Expense Reimbursement": "Questions about reimbursements or expense claims",
    "IT & Access Issues": "Issues about system access, accounts, or technical problems",
    "Events & Celebrations": "Emails about office events, parties, or celebrations"
}

descriptive_labels = list(label_map.values())

# Step 3: Fetch unclassified emails
cursor.execute("""
    SELECT email_id, subject, body, thread_id, received_at
    FROM emails
    WHERE classified_category IS NULL
    ORDER BY received_at;
""")
emails_to_classify = cursor.fetchall()

print(f"🔎 Found {len(emails_to_classify)} unclassified emails.")

# Step 4: Classify and update category + confidence (NO escalation logic yet)
for email_id, subject, body, thread_id, received_at in emails_to_classify:

    # Fetch previous emails for context
    cursor.execute("""
        SELECT subject, body FROM emails
        WHERE thread_id = %s AND received_at < %s
        ORDER BY received_at;
    """, (thread_id, received_at))
    previous_emails = cursor.fetchall()

    context = ""
    for prev_subj, prev_body in previous_emails:
        context += f"Subject: {prev_subj}\nBody: {prev_body}\n---\n"

    email_text = context + f"Subject: {subject}\nBody: {body}"

    result = classifier(email_text, descriptive_labels)
    predicted_description = result["labels"][0]
    confidence = result["scores"][0]

    predicted_label = next(
        key for key, value in label_map.items() if value == predicted_description
    )

    print(f"📧 Email ID {email_id} → Category: {predicted_label}, Confidence: {round(confidence, 2)}")

    # Update category and confidence only
    cursor.execute("""
        UPDATE emails
        SET classified_category = %s,
            classification_confidence = %s
        WHERE email_id = %s;
    """, (predicted_label, confidence, email_id))

conn.commit()
print("✅ Classification and confidence updated for all emails.")


Device set to use cpu


🔎 Found 0 unclassified emails.
✅ Classification and confidence updated for all emails.


In [4]:
threshold = 0.2  # You can change this anytime

# Reset escalation and categories for all classified records
cursor.execute("""
    UPDATE emails
    SET escalated = NULL,
        classified_category = NULL
    WHERE classification_confidence IS NOT NULL;
""")
conn.commit()

# Apply escalation rule: low confidence → escalated = TRUE and category = 'human_intervention'
cursor.execute("""
    UPDATE emails
    SET escalated = CASE
                      WHEN classification_confidence < %s THEN TRUE
                      ELSE FALSE
                  END,
        classified_category = CASE
                                WHEN classification_confidence < %s THEN 'human_intervention'
                                ELSE classified_category
                              END
    WHERE classification_confidence IS NOT NULL;
""", (threshold, threshold))

conn.commit()
print("✅ Escalation flags and categories updated based on confidence threshold.")


✅ Escalation flags and categories updated based on confidence threshold.


In [10]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails where escalated=True;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_52216\28204962.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails where escalated=True;", conn)


,email_id,sender_email,subject,body,received_at,classified_category,status,escalated,thread_id,classification_confidence,response_email,learn
0,139,user39@example.com,Interview reschedule request,I need to reschedule my interview.,2025-03-24 17:52:00.000000,Recruitment Process,pending,True,218.0,0.176617,None,None
1,25,newsletters@audible.com,90% off plus a bonus $20 credit.,audible https://www.audible.com/sign-in?rdpath...,2024-11-27 09:44:15.000000,Events & Celebrations,NOT RESPONDED,True,NaN,0.129796,None,None
2,112,user12@example.com,Negotiating offer terms,I would like to discuss the offered compensati...,2025-03-31 11:32:00.000000,Job Offer,pending,True,206.0,0.179425,None,None
3,27,email@market.temuemail.com,Your loyalty has paid off!,----------------------------------------------...,2024-11-27 01:10:05.000000,Events & Celebrations,NOT RESPONDED,True,NaN,0.190893,None,False
4,137,user37@example.com,Application status update,What is the status of my job application?,2025-03-24 09:24:00.000000,human_intervention,pending,True,217.0,0.163099,None,None
5,140,user40@example.com,Job application withdrawal,I would like to withdraw my application.,2025-03-23 10:56:00.000000,Onboarding,pending,True,219.0,0.143597,None,True
6,34,email@market.temuemail.com,A massive thank you! $6 OFF for you!,----------------------------------------------...,2024-11-20 01:17:09.000000,human_intervention,NOT RESPONDED,True,NaN,0.199713,None,None
7,177,user77@example.com,Assistance required,Im unsure who to contact regarding this issue.,2025-03-31 17:40:00.000000,human_intervention,pending,True,230.0,0.194488,None,None
8,30,email@market.temuemail.com,"Your Purchase, Our Thanks!",----------------------------------------------...,2025-05-04 00:14:43.392800,human_intervention,NOT RESPONDED,True,NaN,0.198693,None,None


In [8]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails where email_id=27;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_52216\1309588657.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails where email_id=27;", conn)


,email_id,sender_email,subject,body,received_at,classified_category,status,escalated,thread_id,classification_confidence,response_email,learn
0,27,email@market.temuemail.com,Your loyalty has paid off!,----------------------------------------------...,2024-11-27 01:10:05,Events & Celebrations,NOT RESPONDED,True,None,0.190893,None,False
